### Air Quality data - data quality assessment

For our specific use (historical analysis, ML model training), the DQ dimensions of interest are completeness, accuracy and consistency.

We are willing to discard timeliness and currency, as for our purpose we prefer having verified data that takes a little longer to get(low currency), and because we are not in a particular hurry to get new values(relaxed timeliness).

Therefore we deem the tradeoff between these dimensions as most useful for our project.

In [1]:
import pandas as pd

In [2]:
aq_data = pd.read_csv('./AQ_APIdata_localtime.csv')

In [3]:
display(aq_data)

,Unnamed: 0,pm10,pm2_5,nitrogen_dioxide,sulphur_dioxide,ozone,localtime
0,0,85.9,76.6,38.5,4.5,2.0,2019-12-31 01:00:00
1,1,84.5,76.9,35.2,4.9,4.0,2019-12-31 02:00:00
2,2,83.0,76.4,35.1,5.1,5.0,2019-12-31 03:00:00
3,3,80.7,74.9,33.5,4.8,2.0,2019-12-31 04:00:00
4,4,77.6,73.1,29.6,3.9,3.0,2019-12-31 05:00:00
...,...,...,...,...,...,...,...
35082,35082,35.9,33.6,26.4,1.8,7.0,2023-12-31 19:00:00
35083,35083,38.1,31.8,27.2,2.0,2.0,2023-12-31 20:00:00
35084,35084,32.1,29.4,31.0,2.0,1.0,2023-12-31 21:00:00
35085,35085,34.6,30.6,32.2,2.2,1.0,2023-12-31 22:00:00


#### Syntactic Accuracy

First, we want to evaluate syntactic accuracy of pollutant values, to do so we:
- Take the lowest and highest possible values from the European Environment Agency's table relative to EAQI level as a range of reference.
    - Ranges of reference can change with respect to pollutants(minimum is always zero)
- Evaluate if any rows in the dataset exceed these thresholds

Unfortunately there is no clear way to measure semantic accuracy, for that we will have to trust the data providers.

In [4]:
def checkRangeOfReference_max(pollutant,maximum,data):
    print("For "+pollutant+" :")
    #This will return the number of rows in the dataset where values exceed the maximum
    return data[data[pollutant] > maximum].shape[0]

In [5]:
pollutants = ['nitrogen_dioxide','ozone',"sulphur_dioxide",'pm2_5','pm10']

Info on thresholds and max/mins is at: https://airindex.eea.europa.eu/AQI/index.html#

In [6]:
maxes = {
    'nitrogen_dioxide': 1000,
    'ozone': 800,
    'sulphur_dioxide': 1250,
    'pm2_5': 800,
    'pm10': 1200
}

In [7]:
for pollutant in pollutants:
    print(checkRangeOfReference_max(pollutant,maxes[pollutant],aq_data))

For nitrogen_dioxide :
0
For ozone :
0
For sulphur_dioxide :
0
For pm2_5 :
0
For pm10 :
0


Let's do the same for the minimum

In [8]:
def checkRangeOfReference_min(pollutant,minimum,data):
    print("For "+pollutant+" :")
    #This will return the number of rows in the dataset where values are below the minimum
    return data[data[pollutant] < minimum].shape[0]

In [9]:
for pollutant in pollutants:
    print(checkRangeOfReference_min(pollutant,0,aq_data))

For nitrogen_dioxide :
0
For ozone :
0
For sulphur_dioxide :
0
For pm2_5 :
0
For pm10 :
0


### Completeness

We plan to verify completeness by:
- Evaluating Row completeness
- Evaluating Column completeness
- Evaluating Dataset completeness
- Evaluating Object completeness by verifying the number of days between time interval and number of rows

In [10]:
# Check number of missing values by column
aq_data.isna().sum()

Unnamed: 0          0
pm10                0
pm2_5               0
nitrogen_dioxide    0
sulphur_dioxide     0
ozone               0
localtime           0
dtype: int64

Given there are no missing values, we can skip te first 3 metrics and mark them as satisfied.

Then, for Object Completeness

In [11]:
# Compute number of days between 31st December 2019 and 31st December 2023
import datetime
#Compute the difference between the two dates, but include them in counting number of days
delta = datetime.date(2023,12,31) - datetime.date(2019,12,31)
# since data is in hourly granularity we should have 24 rows per day
print("For a complete object representation we should have "+str(delta.days*24)+" rows in the dataset")

For a complete object representation we should have 35064 rows in the dataset


In [12]:
aq_data.shape

(35087, 7)

We have 23 bonus hours in the dataset as we moved to local time and have clipped the first hour of 2024, but keep in mind that in aggregation step we will have to compute rolling averages(thats why we kept first rows)

With that said, object completeness could be said to be satisfied(first rolling average will be 23hrs + first hour of 2020, so we don't have leakage of datetime shift to relevant records).

In [13]:
# Checking whether one day appears more than 24 times in the dataset
aq_data["date"] = pd.to_datetime(aq_data["localtime"]).dt.date
aq_data.groupby('date').count().sort_values('localtime',ascending=False).max()

Unnamed: 0          25
pm10                25
pm2_5               25
nitrogen_dioxide    25
sulphur_dioxide     25
ozone               25
localtime           25
dtype: int64

In [14]:
#find the dates that appear more than 24 times
aq_data.groupby('date').count().sort_values('localtime',ascending=False).head()

,Unnamed: 0,pm10,pm2_5,nitrogen_dioxide,sulphur_dioxide,ozone,localtime
date,,,,,,,
2020-10-25,25,25,25,25,25,25,25
2023-10-29,25,25,25,25,25,25,25
2022-10-30,25,25,25,25,25,25,25
2021-10-31,25,25,25,25,25,25,25
2022-08-30,24,24,24,24,24,24,24


The dates appearing 25 times correspond to the days where CEST switched back to CET, and clock were set back by 1h, resulting in 25hr days

Other than those, others have 24 at most

In [15]:
aq_data.groupby('date').count().sort_values('localtime',ascending=True)[:10]

,Unnamed: 0,pm10,pm2_5,nitrogen_dioxide,sulphur_dioxide,ozone,localtime
date,,,,,,,
2019-12-31,23,23,23,23,23,23,23
2020-03-29,23,23,23,23,23,23,23
2021-03-28,23,23,23,23,23,23,23
2022-03-27,23,23,23,23,23,23,23
2023-03-26,23,23,23,23,23,23,23
2022-09-05,24,24,24,24,24,24,24
2022-09-04,24,24,24,24,24,24,24
2022-09-03,24,24,24,24,24,24,24
2022-09-02,24,24,24,24,24,24,24


The same happens for days where CET was switched to CEST (lost an hour) and for 31st of Dec 2019 as we lost a row by moving to localtime

### Consistency

There are two ways we can evaluate consistency:
- Are values intended to have a relationship with other values consistent in said relationship? In other words, given values are assumed to refer to a certain hour of a certain day of a certain year, how many of them are actually consistent in doing so? Hard to verify...
- Are data entries relating to the same type of information (datetimes/timestamps), consistent in upholding the desired format?

We can surely devise a check for the second type of consistency

In [16]:
import re

# Define the regex pattern of our apparent format (yyyy-mm-dd hh:mm:ss)
pattern = r"^(2019|202[0-3])-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01]) ([01]\d|2[0-3]):00:00$"

# Check the number of valid entries in the 'time' column
valid_entries = aq_data['localtime'].apply(lambda x: bool(re.match(pattern, x)))

# Count how many rows match the pattern
num_valid_entries = valid_entries.sum()

In [17]:
print("Number of rows that don't respect our datetime format :", aq_data.shape[0] - num_valid_entries)

Number of rows that don't respect our datetime format : 0


In conclusion, to the extent possible, we have verified that our data satisfies metrics for our data quality dimensions of interest